In [1]:
import pandas as pd
import holidays
from jt import preprocess, loaddf,splitdf


In [2]:
#use plotly for pandas backend
pd.options.plotting.backend = "plotly"
import plotly.io as pio
pio.renderers
pio.renderers.default = "notebook"

In [3]:
df = loaddf()

In [4]:
# use minmaxscaler to scale the data
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()
df['energy'] = scaler.fit_transform(df[['energy']])


In [5]:
df=preprocess(df)

C:\Users\shrey\AppData\Roaming\Python\Python39\site-packages\holidays\countries\india.py:176: Warning:

Diwali and Holi holidays available from 2010 to 2030 only



In [4]:
# df=df.set_index('datetime')

In [5]:

train_df, X_test, y_test = splitdf(df, 100)

In [6]:

india_holidays = holidays.India(years=list(range(2008, 2022)))


C:\Users\shrey\AppData\Roaming\Python\Python39\site-packages\holidays\countries\india.py:176: Warning:

Diwali and Holi holidays available from 2010 to 2030 only



In [7]:
train_df['holiday'] = train_df['datetime'].map(lambda x: int(x in india_holidays))

In [10]:
df['holiday'] = df['datetime'].map(lambda x: int(x in india_holidays))

In [6]:
df.reset_index(inplace=True)
df.columns = ['ds', 'y', 'days_in_month', 'day', 'week_day_of_year', 'day_of_year', 
'holiday', 'week', 'month', 'quarter', 'year', 'dayofyear', 'day_of_week', 'is_month_start', 
'is_month_end', 'is_quarter_start', 'is_quarter_end', 'is_year_start', 'is_year_end', 'is_leap_year',
 'is_weekend', 'sin_week', 'sin_month', 'sin_quarter', 'sin_dayofyear', 'cos_week', 'cos_month', 'cos_quarter', 'cos_dayofyear']


In [ ]:
#xgboost model
import xgboost as xgb
model = xgb.XGBRegressor(objective ='reg:squarederror', colsample_bytree = 0.3, learning_rate = 0.1,
                max_depth = 5, alpha = 10, n_estimators = 10)

In [7]:
from flaml.default import XGBRegressor
model = XGBRegressor()
model.fit(df.drop(['ds', 'y'], axis=1), df['y'])

XGBRegressor(base_score=0.5, booster='gbtree', callbacks=None,
             colsample_bylevel=0.9777437906530106, colsample_bynode=1,
             colsample_bytree=0.9519335125615331, early_stopping_rounds=None,
             enable_categorical=False, eval_metric=None, gamma=0, gpu_id=-1,
             grow_policy='lossguide', importance_type=None,
             interaction_constraints='', learning_rate=0.005485127037677848,
             max_bin=256, max_cat_to_onehot=4, max_delta_step=0, max_depth=0,
             max_leaves=61, min_child_weight=0.01070518287797225, missing=nan,
             monotone_constraints='()', n_estimators=5611, n_jobs=0,
             num_parallel_tree=1, predictor='auto', random_state=0,
             reg_alpha=0.03621564207188963, reg_lambda=1.8045765669466283, ...)

In [8]:
from prophet import Prophet

# train_df = df.copy()
# train_df.reset_index(inplace=True)
# df.columns = ['ds', 'y','holidays']
m = Prophet(daily_seasonality=True,yearly_seasonality=True)
m.fit(df)
# preds = m.predict(train_df, holidays=india_holidays.keys())
# print(preds)
# preds['actual'] = train_df['y']



14:51:17 - cmdstanpy - INFO - Chain [1] start processing
14:54:11 - cmdstanpy - INFO - Chain [1] done processing


In [8]:
test=pd.read_csv('test_WudNWDM.csv')
testdf = test.drop(['row_id'], axis=1)
testdf['datetime'] = pd.to_datetime(testdf['datetime'])
# testdf.columns = ['ds']
testdf = preprocess(testdf)
testdf.reset_index(inplace=True)
testdf.columns = ['ds', 'days_in_month', 'day', 'week_day_of_year', 'day_of_year',
              'holiday', 'week', 'month', 'quarter', 'year', 'dayofyear', 'day_of_week', 'is_month_start',
              'is_month_end', 'is_quarter_start', 'is_quarter_end', 'is_year_start', 'is_year_end', 'is_leap_year',
              'is_weekend', 'sin_week', 'sin_month', 'sin_quarter', 'sin_dayofyear', 'cos_week', 'cos_month', 'cos_quarter', 'cos_dayofyear']


In [9]:
ans = model.predict(testdf.drop(['ds'], axis=1))

In [12]:
test['energy'] = scaler.inverse_transform(ans.reshape(-1,1))
test[['row_id', 'energy']].to_csv('submission11.csv', index=False)


In [11]:
ans = m.predict(testdf)


In [12]:
# test['energy'] = ans['yhat']
test['energy'] = scaler.inverse_transform(ans[['yhat']])
test[['row_id', 'energy']].to_csv('submission10.csv', index=False)
